In [2]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

import pandas as pd
from os.path import join
import numpy as np
from __future__ import division

path = ''

train = pd.read_csv(join(path,'train.csv'),
                    dtype={'is_booking':bool,'srch_destination_id':np.int32, 'hotel_cluster':np.int32},
                    usecols=['srch_destination_id','is_booking','hotel_cluster'],#is_booking is one of the feature chosen here
                    chunksize=1000000)
aggs = []

In [4]:
train[0].head()

TypeError: 'TextFileReader' object does not support indexing

In [5]:
#count the number of rows and number of bookings for every destination-hotel cluster combination.
for chunk in train:
    agg = chunk.groupby(['srch_destination_id',
                         'hotel_cluster'])['is_booking'].agg(['sum','count'])
    agg.reset_index(inplace=True)
    aggs.append(agg)
print('')
aggs = pd.concat(aggs, axis=0) #concatenating will bring duplications on first two columns
aggs.head()

,srch_destination_id,hotel_cluster,sum,count
0,1,20,0.0,2
1,1,30,0.0,1
2,1,60,0.0,2
3,4,22,1.0,2
4,4,25,1.0,2


In [6]:
# Aggregate again to compute the total number of bookings over all chunks.
# Compute the number of clicks by subtracting the number of bookings from total row counts.

NUMBER_OF_CLICKS_FOR_BOOKING = 20
CLICK_WEIGHT =1/NUMBER_OF_CLICKS_FOR_BOOKING

In [7]:
### consider doing a linear regression to find this value

agg = aggs.groupby(['srch_destination_id','hotel_cluster']).sum().reset_index()
agg['count'] -= agg['sum']#click without booking
agg = agg.rename(columns={'sum':'bookings','count':'clicks'})
agg['relevance'] = agg['bookings'] + CLICK_WEIGHT * agg['clicks'] #here can apply linear regression
agg.head()

,srch_destination_id,hotel_cluster,bookings,clicks,relevance
0,0,3,0.0,2.0,0.10
1,1,20,4.0,22.0,5.10
2,1,30,2.0,20.0,3.00
3,1,57,0.0,1.0,0.05
4,1,60,0.0,17.0,0.85


In [8]:
def most_popular(group, n_max=5):
    relevance = group['relevance'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets
    

In [9]:
most_pop = agg.groupby(['srch_destination_id']).apply(most_popular)
most_pop = pd.DataFrame(most_pop).rename(columns={0:'hotel_cluster'})
most_pop.head()

,hotel_cluster
srch_destination_id,
0,3
1,20 30 60 57
2,20 30 53 46 41
3,53 60
4,82 25 32 58 78


In [12]:
test = pd.read_csv(join(path,'test.csv'),
                    dtype={'srch_destination_id':np.int32},
                    usecols=['srch_destination_id'],)
                    
test = test.merge(most_pop, how='left',left_on='srch_destination_id',right_index=True)
test.head()
                    

,srch_destination_id,hotel_cluster
0,12243,5 55 37 11 22
1,14474,5
2,11353,0 31 77 91 96
3,8250,1 45 79 24 54
4,11812,91 42 2 48 59


In [13]:
most_pop_all = agg.groupby('hotel_cluster')['relevance'].sum().nlargest(5).index
most_pop_all = np.array_str(most_pop_all)[1:-1]

test.hotel_cluster.fillna(most_pop_all,inplace=True)

test.hotel_cluster.to_csv(join(path,'predicted_with_pandas.csv'),header=True, index_label='id')